# Data Source

In [0]:
%%shell

git clone https://github.com/SmartyPants042/NER-Deep-Learning.git
pwd
cd NER-Deep-Learning/Data/
unzip dataset.zip
unzip test_dataset.zip

Cloning into 'NER-Deep-Learning'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 48 (delta 8), reused 5 (delta 2), pack-reused 26
Unpacking objects: 100% (48/48), done.
/content
Archive:  dataset.zip
  inflating: dataset.csv             
Archive:  test_dataset.zip
  inflating: test_dataset.csv        


In [0]:
data_input = '/content/NER-Deep-Learning/Data/dataset.csv'
test_input = '/content/NER-Deep-Learning/Data/test_dataset.csv'

In [0]:
# Dataframe manipulations
import pandas as pd
# Array manipulations
import numpy as np

# Library used for deep learning
import tensorflow as tf
# Not all sentences are of same length, padding is required
from tensorflow.keras.preprocessing.sequence import pad_sequences
# Converts the target labels to categories that the neural net can predict
from tensorflow.keras.utils import to_categorical
# Type of model used for DL
from tensorflow.keras.models import Sequential
# Layers present in the network. 
# Refer README.md for more details.
# Analysis of different combinations of layers in README.md
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional

# `Data Manipulation`

In [0]:
df = pd.read_csv(data_input)
df_test = pd.read_csv(test_input)

In [0]:
aggregate_function = lambda s: [(w, p, t) for w, p, t in zip(
    list(s['Word'].values),
    list(s['POS Tag'].values),
    list(s['NER Tag'].values)
)]

In [0]:
sentences = df.groupby("Sentence ID").apply(aggregate_function)
sentences_test = df_test.groupby("Sentence ID").apply(aggregate_function)

In [0]:
max_length = max(len(s) for s in sentences)

In [0]:
words = list(set(df["Word"].values))
words.append("__PAD__")
n_words = len(words)

In [0]:
tags = list(set(df["NER Tag"].values))
n_tags = len(tags)

In [0]:
word2id = {w: i for i, w in enumerate(words)}
tag2id = {t: i for i, t in enumerate(tags)}

# `Training & Testing`

In [0]:
# Creates vector of sentences, where each sentence is itself a vector of 62 words maximum.
# We have not yet converted the words to thier respective IDs.
# We have not yet made the sentences of the same length also known as padding.
X_train_sent = [[tup[0] for i, tup in enumerate(sent) if i<max_length] for sent in sentences]
y_train_sent = [[tup[2] for i, tup in enumerate(sent) if i<max_length] for sent in sentences]
X_test_sent = [[tup[0] for i, tup in enumerate(sent) if i<max_length] for sent in sentences_test]
y_test_sent = [[tup[2] for i, tup in enumerate(sent) if i<max_length] for sent in sentences_test]

In [0]:
def generate_encodings(X_sent, y_sent):
    """
    Description: 
    Converts the list of sentences containing words to a list of sentences conataining just numbers.
    If the word is present in the vocabulary, it is assigned the correct corresponding id;
    If the word is present in the twitter data, but not in the GMB data,
    we simply assign it the value of '__PAD__'.
    
    Returns: Two tuple of encoded sentences, encoded target labels
    
    Input Params: Two tuple of sentences and target labels.
    """
    X_train = []
    y_train = []
    
    for x_s, y_s in zip(X_sent, y_sent):

        temp_x = []
        temp_y = []

        for x, y in zip(x_s, y_s):
            try:
                x = word2id[x]
            except:
                x = word2id['__PAD__']
            try:
                y = tag2id[y]
            except:
                y = tag2id['O']

            temp_x.append(x)
            temp_y.append(y)
        
        temp_x = np.array(temp_x)
        temp_y = np.array(temp_y)
        
        X_train.append(temp_x)
        y_train.append(temp_y)
    
    X_train = np.array(X_train)
    y_train = np.array(y_train)

    return (X_train, y_train)

In [0]:
X_train, y_train = generate_encodings(X_train_sent, y_train_sent)
X_test, y_test = generate_encodings(X_test_sent, y_test_sent)

In [0]:
# We make the sentences and the target labels of each of the same length, 62.
X_train = pad_sequences(X_train, maxlen=max_length, padding='post', value=word2id['__PAD__'])
y_train = pad_sequences(y_train, maxlen=max_length, padding='post', value=tag2id['O'])
X_test = pad_sequences(X_test, maxlen=max_length, padding='post', value=word2id['__PAD__'])
y_test = pad_sequences(y_test, maxlen=max_length, padding='post', value=tag2id['O'])

In [0]:
y_train = [to_categorical(i, n_tags) for i in y_train]
y_test = [to_categorical(i, n_tags) for i in y_test]

In [0]:
model = Sequential([
                    # Takes the ids of words and returns the associated vectors
                    # the dimensions of the resulting vector is 64
                    # Dimension Experimentable
                    Embedding(input_dim=n_words, output_dim=64),
                    # The dropout layer takes a random 10% of the nodes
                    # and disconnects it. This prevents overfitting
                    # EXPERIMENTABLE
                    Dropout(0.1),
                    # A bidirectional wrapper over LSTM Layer
                    # This helps capture contexts in both directions
                    Bidirectional(LSTM(
                        # The number of units in LSTM Layer
                        # EXPERIMENTABLE
                        128,
                        # The Activation function can be expiremented with.
                        # Try using 'sigmoid' function or the more famous,
                        # 'relu' functions
                        # EXPERIMENTABLE
                        activation='tanh',
                        # This layer returns the full output.
                        # If not present, just returns the last output 
                        return_sequences=True, 
                        # EXPERIMENTABLE
                        recurrent_activation='sigmoid', 
                        use_bias=True,
                        )),
                    # keep one-to-one relations on input and output
                    # and lets sequence map indipendently
                    TimeDistributed(Dense(
                        # The number of possible 'buckets' we can put the final
                        # predicted tag into
                        n_tags,
                        # Used to calculate probability distributions,
                        # can not be experimented with without major loss of 
                        # logic and accuracy
                        activation='softmax'
                    ))
])

In [0]:
# Describes the layers in the network
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          3319552   
_________________________________________________________________
dropout (Dropout)            (None, None, 64)          0         
_________________________________________________________________
bidirectional (Bidirectional (None, None, 256)         197632    
_________________________________________________________________
time_distributed (TimeDistri (None, None, 5)           1285      
Total params: 3,518,469
Trainable params: 3,518,469
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(
    # Can be experimented by replacing with 'adam' 
    # Both show almost the same result and take almost same training time
    # EXPERIMENTABLE
    optimizer='rmsprop',
    # The way to measure how accurately the network has predicted the tag
    # from the given n_tags
    # EXPERIMENTABLE
    loss='categorical_crossentropy',
    # We will measure performance on the basis of accuracy
    # EXPERIMENTABLE
    metrics=['accuracy'])

In [0]:
history = model.fit(
    #  The Training data: Features and Labels
    X_train, np.array(y_train),
    #  The number of training examples in one forward/backward pass
    # EXPERIMENTABLE
    batch_size=256,
    # The number of iterations over the training data
    # EXPERIMENTABLE
    epochs=2,
    # Shows the details of training
    verbose=1,
    # Shuffles the training data
    # so that each time the network sees a new sequence of batches
    shuffle=True,
    # Splitting the Total Data and validating on 25% of the data
    # EXPERIMENTABLE
    validation_split=0.25,
    # Some machine level optimizations
    # EXPERIMENTABLE
    workers=1,
    use_multiprocessing=False
)

Epoch 1/2
1553/1553 [==============================] - 448s 289ms/step - loss: 0.0083 - accuracy: 0.9979 - val_loss: 1.4244e-04 - val_accuracy: 1.0000
Epoch 2/2
1553/1553 [==============================] - 455s 293ms/step - loss: 1.4331e-04 - accuracy: 1.0000 - val_loss: 9.8809e-05 - val_accuracy: 1.0000
